In [1]:
# %load imports
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.chrome.options import Options as ChromeOptions
from datetime import datetime
CHROME_DRIVER = './chromedriver'
FIREFOX_DRIVER = './geckodriver'
import time
import re
import requests
import json
import random
import copy
import pprint


def setup_driver(headless=True, driver_type=CHROME_DRIVER) -> webdriver:
    if 'chrome' in driver_type.lower():
        options = ChromeOptions()
        options.add_argument('--ignore-certificate-errors')
        options.add_argument('--ignore-ssl-errors')
        if headless:
            options.add_argument('-headless')
        driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)
    else:
        options = FirefoxOptions()
        if headless:
            options.add_argument('-headless')
        driver = webdriver.Firefox(executable_path=driver_type, firefox_options=options)
    return driver


In [2]:
def get_soup(link):
    driver = setup_driver()
    driver.get(link)
    time.sleep(8)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    executing_query = "document.getElementsByClassName('popover-x-button-close icl-CloseButton')[0].click()"
    if soup.find('div',{'id':'popover-form-container'}):
        driver.execute_script(executing_query)
    print(driver.current_url)
    driver.quit()
    return soup


def parse_all_pages(page):
    page_links = []
    blocks = page.find('ul',{'class':'pagination-list'}).find_all('li')
    for block in blocks:
        if block.find('a', href = True):
            incomplete_link = block.find('a')['href']
            page_links.append('https://www.indeed.com'+incomplete_link)
    pages = []
    pages.append(page)
    for link in set(page_links):
        pages.append(get_soup(link))
    return pages

def get_all_soups(pages):
    soups = []
    links_to_postings = []
    for page in pages:
        try:
            container = page.find('div',{'class':'mosaic mosaic-provider-jobcards mosaic-provider-hydrated'})
            boxes = container.find_all('a', href = True)
        except:
            container = page.find('div',{'class':'jobsearch-SerpJobCard unifiedRow row result clickcard'})
            boxes = container.find_all('a', href = True)
        for box in boxes:
            first = box['href']
            if '/rc/clk?jk' in first:
                link = first.replace('/rc/clk?','https://www.indeed.com/viewjob?')
                links_to_postings.append(link)
    for link in set(links_to_postings):
        soups.append(get_soup(link))
    return soups

In [3]:
page = get_soup('https://www.indeed.com/jobs?q=%22Software+Engineer%22&l=')

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


https://www.indeed.com/jobs?q=%22Software+Engineer%22


In [4]:
pages = parse_all_pages(page)

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=40
https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=10
https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=30
https://www.indeed.com/jobs?q=%22Software+Engineer%22&start=20


In [ ]:
soups = get_all_soups(pages)

<ipython-input-1-42be7db9a6ee>:26: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path=driver_type, chrome_options=options)


https://www.indeed.com/viewjob?jk=1c1cca2129ef17d6&fccid=9784ae78e9834539&vjs=3
https://www.indeed.com/viewjob?jk=df080646c9d96481&fccid=6824fc1f087bd63e&vjs=3
https://www.indeed.com/viewjob?jk=e2ac69e3df8da482&fccid=5b47cb6dc9bd7f35&vjs=3
https://www.indeed.com/viewjob?jk=10ea647495b66b08&fccid=f766f8bfbc3effb7&vjs=3
https://www.indeed.com/viewjob?jk=d18cde112dad3c0c&fccid=5b2f9eaa91cb3370&vjs=3
https://www.indeed.com/viewjob?jk=69125030be3ca818&fccid=dae005da9b084571&vjs=3
https://www.indeed.com/viewjob?jk=58e2ffa598ee8ccf&fccid=10b5c722d846df43&vjs=3
https://www.indeed.com/viewjob?jk=98cf54f6d023afb3&fccid=a575b7c5c50b587a&vjs=3
https://www.indeed.com/viewjob?jk=f13c90de44845b92&fccid=4ed80f3a97849f22&vjs=3
https://www.indeed.com/viewjob?jk=835c0973db3a626b&fccid=6bcbaf35af26e1a6&vjs=3
https://www.indeed.com/viewjob?jk=86a75dfd51f0656c&fccid=8c805653641d337d&vjs=3
https://www.indeed.com/viewjob?jk=d5c9b65ffefc69ba&fccid=bee6abba453010ff&vjs=3
https://www.indeed.com/viewjob?jk=94bd47

In [ ]:
sd = page.find('div',{'class':'jobsearch-SerpJobCard unifiedRow row result clickcard'})

In [ ]:
for s in sd:
    d = s.find('a', href = True)
    print(d['href'])

In [ ]:
container = page.find('div',{'class':'mosaic mosaic-provider-jobcards mosaic-provider-hydrated'})

In [ ]:
boxes = container.find_all('a', href =True)

In [ ]:
boxes[0]['href'

In [ ]:
links_to_postings = []
for box in boxes:
    first = box['href']
    if '/rc/clk?jk' in first:
        link = first.replace('/rc/clk?','https://www.indeed.com/viewjob?')
        links_to_postings.append(link)

In [ ]:
soups = []
for link in links_to_postings:
    soup = get_soup(link)
    soups.append(soup)

In [ ]:
for soup in soups:
    title = soup.find('h1',{'class':'icl-u-xs-mb--xs icl-u-xs-mt--none jobsearch-JobInfoHeader-title'}).text
    
    original = soup.find('div',{'id':'originalJobLinkContainer'}).find('a')['href']
    print(original)